In [149]:
from importlib import reload
import sys
import util
reload(util)
data = util.FontData
FontDataset = util.FontDataset
import experiment
reload(experiment)
from sklearn import linear_model, neural_network

In [150]:
data.load()

Loading embeddings...done
Loading typographic + semantic vectors...done
Loading images...done
Loading glyphs...done
Loading SVGs...done


In [151]:
train_dataset = FontDataset(data, 'train')
val_dataset = FontDataset(data, 'val')

In [152]:
train_typographic = data.get_all_typographic('train')
val_typographic = data.get_all_typographic('val')

In [153]:
train_image = data.get_all_image('train')
train_image = train_image.reshape(train_image.shape[0], -1)
val_image = data.get_all_image('val')
val_image = val_image.reshape(val_image.shape[0], -1)

In [154]:
train_semantic = data.get_all_semantic('train')
val_semantic = data.get_all_semantic('val')

### Linear Regression: Image -> Typographic

In [155]:
image_typographic_reg = linear_model.LinearRegression()

In [156]:
image_typographic_reg.fit(train_image, train_typographic)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [157]:
train_stats, train_typographic_pred = experiment.run_experiment(image_typographic_reg, train_image, train_typographic)

{'RMSE': 1.3667070561496075e-05, 'R2': 0.9999999998025613}


In [158]:
val_stats, val_typographic_pred = experiment.run_experiment(image_typographic_reg, val_image, val_typographic)

{'RMSE': 1.3457238674163818, 'R2': -0.6625254045401793}


### Linear Regression: Typographic -> Semantic

In [159]:
typographic_semantic_reg = linear_model.LinearRegression()

In [160]:
typographic_semantic_reg.fit(train_typographic, train_semantic)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [161]:
train_stats, train_semantic_pred = experiment.run_experiment(typographic_semantic_reg, train_typographic, train_semantic)

{'RMSE': 0.6425207853317261, 'R2': 0.5791966258956402}


In [162]:
val_stats, val_semantic_pred = experiment.run_experiment(typographic_semantic_reg, val_typographic, val_semantic)

{'RMSE': 0.652705729007721, 'R2': 0.5643953306184701}


### Linear Regressions: Image -> Typographic -> Semantic

In [163]:
class SequentialLinearRegression:
    def __init__(self, reg1, reg2):
        self.reg1 = reg1
        self.reg2 = reg2
    def predict(self, X):
        return self.reg2.predict(self.reg1.predict(X))

image_typographic_semantic_reg = SequentialLinearRegression(image_typographic_reg, typographic_semantic_reg)

In [164]:
train_stats, train_semantic_pred = experiment.run_experiment(image_typographic_semantic_reg, train_image, train_semantic)

{'RMSE': 0.6425206065177917, 'R2': 0.5791968681664224}


In [165]:
val_stats, val_semantic_pred = experiment.run_experiment(image_typographic_semantic_reg, val_image, val_semantic)

{'RMSE': 1.2455190420150757, 'R2': -0.617107781671052}


### Linear Regression: Image -> Semantic

In [166]:
image_semantic_reg = linear_model.LinearRegression()

In [167]:
image_semantic_reg.fit(train_image, train_semantic)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [168]:
train_stats, train_semantic_pred = experiment.run_experiment(image_semantic_reg, train_image, train_semantic)

{'RMSE': 1.5096046809048858e-05, 'R2': 0.9999999997676309}


In [169]:
val_stats, val_semantic_pred = experiment.run_experiment(image_semantic_reg, val_image, val_semantic)

{'RMSE': 1.4704996347427368, 'R2': -1.250469831432338}


### Multilayer Perceptron: Image -> Semantic

In [170]:
image_semantic_nn = neural_network.MLPRegressor(hidden_layer_sizes=(10,))

In [171]:
image_semantic_nn.fit(train_image, train_semantic)

MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
             beta_2=0.999, early_stopping=False, epsilon=1e-08,
             hidden_layer_sizes=(10,), learning_rate='constant',
             learning_rate_init=0.001, max_iter=200, momentum=0.9,
             n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
             random_state=None, shuffle=True, solver='adam', tol=0.0001,
             validation_fraction=0.1, verbose=False, warm_start=False)

In [172]:
train_stats, train_semantic_pred = experiment.run_experiment(image_semantic_nn, train_image, train_semantic)

{'RMSE': 0.9919090270996094, 'R2': -0.0038887246173080624}


In [173]:
val_stats, val_semantic_pred = experiment.run_experiment(image_semantic_nn, val_image, val_semantic)

{'RMSE': 0.9884147047996521, 'R2': -0.004764330244239873}


#### Example

In [174]:
val_semantic_pred[0]

tensor([ 0.0473,  0.0245, -0.0761,  0.0117, -0.1122,  0.0179,  0.0777,  0.0049,
         0.1843, -0.0787, -0.0834,  0.0243,  0.0282, -0.0360, -0.0476,  0.0211,
        -0.0109, -0.0849,  0.0214, -0.0614,  0.0274,  0.0110,  0.1281, -0.0314,
        -0.0065, -0.1098,  0.0356,  0.0109, -0.0034, -0.0432, -0.0155])

In [175]:
data.get_semantic(data.get_name(0, "val"))

array([-0.29138137, -0.16184933, -0.14702909, -0.40446154,  0.43245715,
        0.5095358 , -0.62126273, -0.99704623, -0.28945408, -0.07533271,
       -0.63554327, -0.19085355,  0.09324976, -0.46094253,  0.13544815,
       -0.91632764, -0.45007817, -0.93885684, -0.88615512, -0.3845307 ,
        0.30207304, -0.16686988, -0.25102446,  0.14758578, -0.66951492,
       -0.74703117,  0.53204622,  0.61595246, -0.65247733,  0.21868042,
        0.17455805])